<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/20240518_0932_DNA_Study_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl